<div align="right">Naoaki ONO, Shigehiko KANAYA <br/>
NAIST [Data Science Center](http://www-dsc.naist.jp/dsc_jp/)</div>

[Open in Google Colaboratory](https://colab.research.google.com/github/naono-git/colaboratory/blob/master/note_05_hello_RDKit.ipynb)

#はじめてのRDKit

ケモインフォマティクスの分野では化学分子の複雑な構造を記号化して計算機上で扱うために、さまざまな手法が提案されてきています。

[RDKit](https://www.rdkit.org)はオープンソースで開発されているケモインフォマティクス関連のツールキットで、Pythonからも利用することができます。

Google Colab上で使えるPython環境にはRDKitはインストールされていないのですが、幸い、Colabでは自分が必要とするライブラリを追加でインストールして利用することが許されています。

ただし、あくまでもColab内で一時的に作られた仮想マシン上にインストールするだけなので、再接続などの場合も含め、Colabを起動するたびに**毎回インストールしなければなりません**。

RDKitの場合、関連するライブラリをいろいろインストールしないといけないので、全部をインストールには3、4分かかります。将来的にはもう少し簡略化されるかもしれませんが、現状は我慢せざるを得ません。実際に研究などに使う場合はTensorFlow環境も含めて自分の計算機にセットアップする方が現実的かもしれませんが、とりあえず試しに使ってみる程度なら問題ないでしょう。

## RDKitのインストール

とりあえずおまじないと思って以下の3つのセルを実行して下さい。

3つ目のセルの実行結果で

 A module for molecules and stuff

 see Chem/index.html in the doc tree for documentation

と表示されれば問題ありません。
もしうまくいかないようならこのページをリロードしてからやり直してみて下さい。

In [0]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

In [0]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [0]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
# from rdkit import Chem

print(Chem.__doc__)

インストールが済んだら、次はサンプルデータをダウンロードしましょう。
ケモインフォマティクスの練習データとして定番の、Tox21データセットをダウンロードしてきます。




In [0]:
!curl https://raw.githubusercontent.com/pdcoded/fox/master/tox21_sdf/tox21_10k_data_all.sdf -O

sdf(Structure Data File)形式と言われるデータファイルですが、実のところ中身は下記のように、

原子の種類と位置

原子間の結合グラフ

特性値など

が分子ごとに順番に記録された単なるテキストファイルです。


In [0]:
#SDF file to mol and Dataframe
def sdf_to_df(filepass):
    mols = [mol for mol in Chem.SDMolSupplier(filepass) if mol is not None]
    for id, mol in enumerate(mols):
        if id == 0:
            dicts = mol.GetPropsAsDict()
            df = pd.DataFrame(dicts, index=[id,])
        else:
            dicts = mol.GetPropsAsDict()
            dfplus = pd.DataFrame(dicts, index=[id,])
            df = df.append(dfplus)
    return mols, df

train_x, train_df = sdf_to_df('./tox21_10k_data_all.sdf')

ようやく準備ができました。

データの個数を確認しておきます。

In [0]:
len(train_x)

分子構造はMolオブジェクトという構造体になっています。

In [0]:
train_x[0]

Molオブジェクトから分子式やSMILESなど、さまざまなデータ構造に自動で変換する関数が揃っています。

In [0]:
mol1 = train_x[1]
#print(Chem.MolToMolBlock(mol1))
print(Chem.MolToSmiles(mol1))

In [0]:
from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D
# SVG(Chem.MolToSVG(mol1))
drawer = rdMolDraw2D.MolDraw2DSVG(300,300)
drawer.DrawMolecule(mol1)
drawer.FinishDrawing()
svg = drawer.GetDrawingText()
SVG(svg)

Tox21の教師データの方ですが、ストレス応答タンパクとの相互作用など、12種類の生化学アッセイの定量結果が各分子について記録されています（全部の分子について全部のテストが行われているわけではありません、というかむしろテストされている方が少ない）。

In [0]:
train_df.columns

In [0]:
import matplotlib.pyplot as plt

教師データの配列をマップ化してみると、データなし（灰色）がほとんどで作用あり（黒）や作用なし（白）が10％前後であることがわかります。

In [0]:
hoge = train_df.iloc[0:20,3:15].as_matrix()
dumb = plt.imshow(hoge*255,extent=[0,12,0,20])

評価値をどれか一つ選んでNNで回帰してみましょう。

上図でみるように、割とデータが疎なので、多少なりとも相互作用しているリガンドが多い方がいいでしょうか。

In [0]:
#print(np.mean(train_df['NR-AR']))
#print(np.mean(train_df['NR-AR-LBD']))
#print(np.mean(train_df['NR-AhR']))
#print(np.mean(train_df['NR-Aromatase']))
#print(np.mean(train_df['NR-ER']))
#print(np.mean(train_df['NR-ER-LBD']))
#print(np.mean(train_df['NR-PPAR-gamma']))
#print(np.mean(train_df['SR-ARE']))
#print(np.mean(train_df['SR-ATAD5']))
#print(np.mean(train_df['SR-HSE']))
print(np.mean(train_df['SR-MMP']))  ## Small Molecule Disruptors Of The Mitochondrial Membrane Potential
#print(np.mean(train_df['SR-p53']))

In [0]:
plt.plot(train_df['SR-MMP'])

ミトコンドリアの膜電位の消失をトリガーとしてアポトーシスが働くという機構があるため、MMPとの相互作用は細胞の生死に大きな影響を与える、らしいです。

note_03と同様の全結合ニューラルネットワークを使って、分子フィンガープリントからMMPの活性を予測してみましょう。

In [0]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Dropout, Activation

Morgan FingerPrintは各原子の近傍の原子の種類を数え上げることで分子の構造を定量化する、分子構造の特徴量モデルのなかでもスタンダードなものの一つです。

In [0]:
from rdkit.Chem import AllChem
n0 = 1000
n_train = 8000
n_test = 2000

tmp = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024) for mol in train_x[n0:(n0+n_train)]]
x_train = np.stack(tmp,0)
y_train = train_df['SR-MMP'][n0:(n0+n_train)]
y_train[np.isnan(y_train)] = 0.5

tmp = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024) for mol in train_x[(n0+n_train):(n0+n_train+n_test)]]
x_test = np.stack(tmp,0)
y_test = train_df['SR-MMP'][(n0+n_train):(n0+n_train+n_test)]
y_test[np.isnan(y_test)] = 0.5

In [0]:
print(np.mean(y_train))
print(np.mean(y_test))

三層の全結合ネットワークを介して1ノードの出力としてMMPの活性を予測します。
最初の層のパラメーター数が若干過剰なので、念の為Dropoutレイヤーを追加してあります。

In [0]:
model1 = Sequential()
model1.add(Dense(16, input_shape=(1024,), activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(1, activation='relu'))
model1.summary()

In [0]:
model1.compile(loss='mean_squared_error',  optimizer='adagrad', metrics=['accuracy'])

In [0]:
hist1 = model1.fit(x=x_train, y=y_train, verbose=0, epochs=100,  shuffle=True, validation_data=(x_test, y_test))
model1.evaluate(x=x_test,y=y_test)

ざっくり100エポック学習させたところ、97%程度の制度で予測できることがわかりました。

In [0]:
hoge = model1.predict(x_test)
np.mean(hoge)

ただ、testデータでのaccuracyが下がっていく傾向が続いているので、過学習の疑いが若干残ります。
しっかりした予測のためにはデータ数やネットワークの構造を検証する必要があるかもしれません。

In [0]:
plt.plot(hist1.history['acc'])
plt.plot(hist1.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(hist1.history['loss'])
plt.plot(hist1.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

講義の方で紹介したGraph Convolutional Neural Networkについては、Google Colaboratory上で必要なライブラリをインストールするのが若干難しいため、実習は割愛させていただきます。

自前で環境を構築できる方は、Dockerなどを利用してDeepChemライブラリーをインストールして、GCNNを試してみて下さい。

# おわりに
深層学習のモデルは次々と新しいものが開発され、いま流行っているものもおそらく3、4年後には時代遅れになってしまうものと思われます。個々のプログラムをフォローすることよりも、基本となる概念を把握しておくことが大事になってくることでしょう。

みなさんが面白いモデルを開発されていくことを祈念して、結びに替えせていただきます。


以上で深層学習入門を終了といたします。

お疲れ様でした。